In [1]:
# Iniciar drive
import pandas as pd
import time
import datetime
import boto3
import random
import os
from urllib.request import urlretrieve

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By

#binary = r'C:\Users\pedro.costa\AppData\Local\Mozilla Firefox\firefox.exe'
binary = r'C:\Program Files\Mozilla Firefox\firefox.exe'

#options = Options()
#options.set_headless(headless=False)
#options.binary = binary
#cap = DesiredCapabilities.FIREFOX
#cap["marionette"] = True #optional
#driver = webdriver.Firefox(firefox_options=options, capabilities=cap, executable_path="C:\\Dropbox (Indie Capital)\\Sistemas\\Files\\CVM\\Informe diario\\geckodriver.exe")

driver = webdriver.Firefox(executable_path="C:\\Helena\\geckodriver\\geckodriver.exe")
url = "http://bravi.tv/associados/"
driver.get(url)

#driver.add_cookie({"name":"cookiesPreferencesNotLogged","domain":".mercadolivre.com.br","value":"%7B%22categories%22%3A%7B%22advertising%22%3Atrue%7D%7D"})
#driver.add_cookie({"name":"cp","domain":".mercadolivre.com.br","value":usar_cep + "%7C1649687491347"})
#driver.add_cookie({"name":"modal-configuration","domain":".mercadolivre.com.br","value":'{"full_modal_fs":{"view_cnt":2,"close_cnt":2,"view_time":1649689874,"close_time":1649689883}}'})
#driver.get("https://lista.mercadolivre.com.br/acessorios-veiculos/caminhoes/outros/")

In [22]:
def exeClick(ndiv):
    path = '/html/body/div[2]/div[2]/div[1]/div/div['+ str(ndiv) +']/a/div/img'
    driver.find_element(By.XPATH, path).click()
    time.sleep(2)

def getInfo():
    path = '/html/body/div[2]/div[2]/div[1]/div/div[4]/div/div/div[2]/div[1]'
    info = driver.find_element(By.XPATH, path).text
    return info.replace(",","")

def getSite():
    path = '/html/body/div[2]/div[2]/div[1]/div/div[4]/div/div/div[2]/div[2]/a[1]'
    site = driver.find_element(By.XPATH, path).get_attribute('href')
    return site

def getEmail():
    path = '/html/body/div[2]/div[2]/div[1]/div/div[4]/div/div/div[2]/div[2]/a[2]'
    email = driver.find_element(By.XPATH, path).get_attribute('href')
    
    return email.replace("mailto:","")

def getAssociado():
    path = '/html/body/div[2]/div[2]/div[1]/div/div[4]/div/h3'
    associado = driver.find_element(By.XPATH, path).text
    return associado

def sPlanilha(associado,contato,email,site,cidade,endereco,cep):
    with open('dadosBravi.csv', 'a') as dados:
        dados.write(associado+","+contato+","+email+","+site+","+cidade+","+endereco+","+cep+"\n")

def indexOf(linha,busca):
    ret = 0
    try:
        ret = linha.index(busca)
    except:
        ret = 0
    return ret


In [30]:
# CSV FILE HEADER
sPlanilha("Associado","Contato","E-mail","Site","Cidade","Endereco","CEP")


In [55]:
def parseInfo(info):
    endereco = ''
    cep = ''
    cidade = ''
    contato = ''

    linhas = info.split("\n")
    print('estou na funcao')
    print('info: ' + info)
    for x in range(len(linhas)):
        linha = linhas[x]
        print(linha)
        if linha[0:4]=="CEP:":
            cep = linha[5:len(linha)]
            continue
        if linha[0:8]=="Contato:": 
            if contato=="":
                contato = linha[9:len(linha)]
            else:
                contato = contato + " " + linha[9:len(linha)]
            continue
        if indexOf(linha,"/") > 0:
            print("achou a barra")
            cidade = linha
            continue

        match x:
            case 0:
                endereco = linha
            case 1:
                endereco = endereco + " " + linha
            case 2:
                if linha[0:4]!="CEP:":
                    cidade = linha
            case default:
                print("caiu no default")
                if contato=="":
                    contato = linha
                else:
                    contato = contato + " " + linha
    return endereco, cep, cidade, contato

In [ ]:
for ndiv in range(6,664):
    exeClick(ndiv)
    info = getInfo()
    site = getSite()
    email = getEmail()
    associado = getAssociado()

    endereco, cep, cidade, contato = parseInfo(info)

    print("nDiv: "+str(ndiv))
    print("site: "+site)
    print("email: "+email)
    print("========")
    print("endereco: " + endereco)
    print("cep: " + cep)
    print("cidade: " + cidade)
    print("contato: " + contato)
    
    sPlanilha(associado,contato,email,site,cidade,endereco,cep)
print("-- ACABOU --")

estou na funcao
info: João Afonso 40 - Humaitá
CEP: 22.261-040
Rio de Janeiro / RJ
Contato: (21) 2225-7598
João Afonso 40 - Humaitá
CEP: 22.261-040
Rio de Janeiro / RJ
achou a barra
Contato: (21) 2225-7598
nDiv: 6
site: http://www.2molequesproducoes.com.br/
email: ajose2@gmail.com
endereco: João Afonso 40 - Humaitá
cep: 22.261-040
cidade: Rio de Janeiro / RJ
contato: (21) 2225-7598
estou na funcao
info: Rua General Danton Teixeira 87
CEP: 22610-350
Rio de Janeiro / RJ
Contato: (21) 98123-9988
Rua General Danton Teixeira 87
CEP: 22610-350
Rio de Janeiro / RJ
achou a barra
Contato: (21) 98123-9988
nDiv: 7
site: 
email: luduque@doisd.net.br
endereco: Rua General Danton Teixeira 87
cep: 22610-350
cidade: Rio de Janeiro / RJ
contato: (21) 98123-9988
estou na funcao
info: Rua Barão do Flamengo 22 - sala 801 - Flamengo
CEP: 22220-900
Rio de Janeiro/RJ
Contato: André Breitman
(21) 2262-5737
Rua Barão do Flamengo 22 - sala 801 - Flamengo
CEP: 22220-900
Rio de Janeiro/RJ
achou a barra
Contato: A

In [26]:
path = '/html/body/div[2]/div[2]/div[1]/div/div[9]/a/div/img'
#info = driver.find_element(By.XPATH, path).get_attribute('src')
driver.find_element(By.XPATH, path).click()
time.sleep(2)

path = '/html/body/div[2]/div[2]/div[1]/div/div[4]/div/div/div[2]/div[1]'
info = driver.find_element(By.XPATH, path).text

path = '/html/body/div[2]/div[2]/div[1]/div/div[4]/div/div/div[2]/div[2]/a[1]'
site = driver.find_element(By.XPATH, path).get_attribute('href')

path = '/html/body/div[2]/div[2]/div[1]/div/div[4]/div/div/div[2]/div[2]/a[2]'
email = driver.find_element(By.XPATH, path).get_attribute('href')

print("site: "+site)
print("email: "+email)
print("info: "+info)

site: https://www.facebook.com/2FlyComunicacao/info/?ref=page_internal
email: mailto:kelly@2fly.com.br
info: Rua 1.112, nº 394, Ed. Lydia Ferez, Sala 05, Térreo
CEP: 74830-370
Rio de Janeiro/RJ
Contato: Kelly Alves
(62) 3095-2868


In [54]:
info = getInfo()
linhas = info.split("\n")
print(linhas)

endereco = ''
cep = ''
cidade = ''
contato = ''

print ('--------')
for x in range(len(linhas)):
    linha = linhas[x]
    print(linha)
    if linha[0:4]=="CEP:":
        cep = linha[5:len(linha)]
        continue
    if linha[0:8]=="Contato:": 
        if contato=="":
            contato = linha[9:len(linha)]
        else:
            contato = contato + " " + linha[9:len(linha)]
        continue
    if indexOf(linha,"/") > 0:
        print("achou a barra")
        cidade = linha
        continue
    
    match x:
        case 0:
            endereco = linha
        case 1:
            endereco = endereco + " " + linha
        case 2:
            if linha[0:4]!="CEP:":
                cidade = linha
        case default:
            print("caiu no default")
            if contato=="":
                contato = linha
            else:
                contato = contato + " " + linha
    
print("========")
print("endereco: " + endereco)
print("cep: " + cep)
print("cidade: " + cidade)
print("contato: " + contato)

['Rua da Alfândega 35 - Loja 401 – Tech Hub Paço Alfândega - Bairro do Recife', 'CEP: 50.030-030', 'Recife / PE', '(81) 99585-3883', 'Contato: Maurício Corrêa']
--------
Rua da Alfândega 35 - Loja 401 – Tech Hub Paço Alfândega - Bairro do Recife
CEP: 50.030-030
Recife / PE
achou a barra
(81) 99585-3883
caiu no default
Contato: Maurício Corrêa
endereco: Rua da Alfândega 35 - Loja 401 – Tech Hub Paço Alfândega - Bairro do Recife
cep: 50.030-030
cidade: Recife / PE
contato: (81) 99585-3883 Maurício Corrêa


In [28]:
info = getInfo()
quebrado = info.split("\n")
print(quebrado)

endereco = ''
cep = ''
cidade = ''
contato = ''
outro = ''

if "CEP" in quebrado[2]:
    match x:
        case 0:
            endereco = quebrado[x]
        case 1:
            endereco = endereco + ' ' + quebrado[x]
        case 2:
            cidade = quebrado[x]
        case 3:
            contato = quebrado[x][9:len(quebrado[x])]
            
else: 
      match x:
            case 0:
                endereco = quebrado[x]
            case 1:
                endereco = endereco + ' ' + quebrado[x]
            case 2:
                cep = quebrado[x][5:len(quebrado[x])]
            case 3:
                cidade = quebrado[x]
            case 4:
                contato = quebrado[x][9:len(quebrado[x])]
    
print("========")
print("endereco: " + endereco)
print("cep: " + cep)
print("cidade: " + cidade)
print("contato: " + contato)
print("outro: " + outro)

['Rua 1.112 nº 394 Ed. Lydia Ferez Sala 05 Térreo', 'CEP: 74830-370', 'Rio de Janeiro/RJ', 'Contato: Kelly Alves', '(62) 3095-2868']
endereco: 
cep: 
cidade: 
contato: -2868
outro: 


In [34]:
indexOf("Recife / PE","/")


7